In [1]:
from collections import defaultdict
from django.db.models.signals import *
import pandas as pd
from itertools import chain
from tqdm.notebook import tqdm
import datetime
import calendar

In [2]:
def normalize_month(month, year):
    return month + 12*(year - 2020)

In [3]:
normalize_month(12, 2023)

48

In [4]:
# def normalize_day(day, month):
#     return day + 31*(month - 1)

In [5]:
# normalize_day(31,12)

In [6]:
def permissoes_history_user(result_list,mes_ano):
    permissoesUsuarios = {}
    mes_referencia = normalize_month(mes_ano.month, mes_ano.year)
    for a in result_list:
        try:
            if not permissoesUsuarios.get(a.user.get_full_name()):
                permissoesUsuarios[a.user.get_full_name()] = {}
            if isinstance(permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year)),list):
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)].append(a.permissions.name)
            elif permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year)):
                multi_perm_per_month= permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year))
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)] = [multi_perm_per_month,a.permissions.name]
            else:
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)] = a.permissions.name
        except:
            pass
    
    usuariosAtivos = []
    usuariosInativos = []

    for nome, mes in permissoesUsuarios.items():
        meses = sorted(list(mes.keys()))
        item = max(list(mes.keys()))
        cargo_atual = mes[item]

# Se em algum momento do mesmo mês o usuário foi diferente de Inativo ele entra como ativo
        if isinstance(cargo_atual,list):
            for cargo in cargo_atual:
                if cargo !='Inativo' and not any(u for u in usuariosAtivos if nome == u):
                    usuariosAtivos.append(nome)     
# se o cargo atual for diferente de inativo e não for uma lista(o que configura multiplas permissões no mesmo mês) ele é ativo
        elif cargo_atual != 'Inativo' and not isinstance(cargo_atual,list):
            usuariosAtivos.append(nome)
# Se no mes anterior ao ultimo ele foi diferente de inativo e no ultimo mes foi Inativo e
# o mes do cargo_atual for igual ao ultimo mes de referencia(ultimo mês pra coleta) e o nome não estiver nos usuariosAtivos
# ele é ativo
        elif mes[meses[-2]] != 'Inativo' and cargo_atual == 'Inativo' and item == mes_referencia and not any(u for u in usuariosAtivos if nome == u):
            usuariosAtivos.append(nome)       
# se o cargo atual contem a palavra Inativo e não está nos usuariosAtivos então ele é Inativo
        elif 'Inativo' in cargo_atual and nome not in usuariosAtivos:
            usuariosInativos.append(nome)
        else:
            print('Corra pras montanhas')
    
    return usuariosAtivos,usuariosInativos

In [7]:
companies=['Modera - Consórcio Gerencia OAE SP',
  'Modera - Consolare',
  'Modera - Cortel',
  'Modera - Maya',
  'Modera - Velar',
  'Modera - Consórcio ME2',
  'Afirma Engenharia Viária',
  'Pavicolet (Amapá)',
  'Pavicolet (Amazonas)',
  'Entrevias',
  'LCM - Ituporanga',
  'LCM - Gaspar',
  'LCM - Criciúma',
  'Llucena SINFRA',
  'Llucena Vias Urbanas',
  'Minasul',
  'Sinalizavia (Municípios)',
  'Sinaliza Via (Rodovias)',
  'Terrayama - DER Araçuaí',
  'Radar SA Concessões',
  'Radar SA',
  'Construtora Castilho',
  'Construtora Castilho (Ferrovias)',
  'Construtora Castilho (Grupo Pátria)',
  'Construtora Castilho (Prefeitura Curitiba)']
len(companies),companies

(25,
 ['Modera - Consórcio Gerencia OAE SP',
  'Modera - Consolare',
  'Modera - Cortel',
  'Modera - Maya',
  'Modera - Velar',
  'Modera - Consórcio ME2',
  'Afirma Engenharia Viária',
  'Pavicolet (Amapá)',
  'Pavicolet (Amazonas)',
  'Entrevias',
  'LCM - Ituporanga',
  'LCM - Gaspar',
  'LCM - Criciúma',
  'Llucena SINFRA',
  'Llucena Vias Urbanas',
  'Minasul',
  'Sinalizavia (Municípios)',
  'Sinaliza Via (Rodovias)',
  'Terrayama - DER Araçuaí',
  'Radar SA Concessões',
  'Radar SA',
  'Construtora Castilho',
  'Construtora Castilho (Ferrovias)',
  'Construtora Castilho (Grupo Pátria)',
  'Construtora Castilho (Prefeitura Curitiba)'])

In [36]:
for a in companies:
    company=Company.objects.get(name=a)
    print(company.name,company.cnpj)

Modera - Consórcio Gerencia OAE SP 52.724.344/0001-47
Modera - Consolare 28.256.567/0001-42
Modera - Cortel 28.256.567/0001-42
Modera - Maya 28.256.567/0001-42
Modera - Velar 28.256.567/0001-42
Modera - Consórcio ME2 28.256.567/0001-42
Afirma Engenharia Viária 05.205.684/0001-81
Pavicolet (Amapá) 46.791.721/0001-68
Pavicolet (Amazonas) 46.791.721/0001-68
Entrevias 26.664.057/0001-89
LCM - Ituporanga 19.758.842/0001-35
LCM - Gaspar 19.758.842/0001-35
LCM - Criciúma 19.758.842/0004-88
Llucena SINFRA 03.992.929/0001-32
Llucena Vias Urbanas 03.992.929/0001-32
Minasul 37.456.439/0001-33
Sinalizavia (Municípios) 22.221.801/0001-10
Sinaliza Via (Rodovias) 22.221.801/0001-10
Terrayama - DER Araçuaí 21.681.150/0001-88
Radar SA Concessões 34.976.105/0001-00
Radar SA 34.976.105/0001-00
Construtora Castilho 92.779.503/0001-25
Construtora Castilho (Ferrovias) 92.779.503/0001-25
Construtora Castilho (Grupo Pátria) 92.779.503/0001-25
Construtora Castilho (Prefeitura Curitiba) 92.779.503/0001-25


In [8]:
companies=[a for a in Company.objects.filter(name__startswith='Construtora Castilho').values_list('name',flat=True)]
companies

['Construtora Castilho (Ferrovias)',
 'Construtora Castilho (Grupo Pátria)',
 'Construtora Castilho (Ramilândia)',
 'Construtora Castilho',
 'Construtora Castilho (Prefeitura Curitiba)']

In [9]:
# Mês e ano para coleta dos usuários
mes=3
ano=2024
dia = calendar.monthrange(ano, mes)[1]
dia,mes,ano

(31, 3, 2024)

In [10]:
# DATA sempre referente ao mês que se quer coletar nesse formato history_date__lte='2024-01-31 03:00:00'
# é utilizado 3 horas da manha pois é a pra fechar com o horario de meio noite do servidor
data = datetime.datetime(ano,mes,dia,3,0,0)
data

datetime.datetime(2024, 3, 31, 3, 0)

In [11]:
pd.DataFrame().to_excel('Usuários Ativos Financeiro.xlsx')

In [12]:
for company in tqdm(companies):
    if company in ['Modera - Consórcio Gerencia OAE SP']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Modera - Consórcio Gerencia OAE SP'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Modera - Consolare','Modera - Cortel','Modera - Maya','Modera - Velar','Modera - Consórcio ME2']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Modera - Consolare','Modera - Cortel','Modera - Maya','Modera - Velar','Modera - Consórcio ME2'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Afirma Engenharia Viária']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Afirma Engenharia Viária'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Pavicolet (Amapá)','Pavicolet (Amazonas)']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Pavicolet (Amapá)','Pavicolet (Amazonas)'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['LCM - Ituporanga','LCM - Gaspar']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['LCM - Ituporanga','LCM - Gaspar'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['LCM - Criciúma']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['LCM - Criciúma'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Llucena SINFRA','Llucena Vias Urbanas']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Llucena SINFRA','Llucena Vias Urbanas'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Minasul']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Minasul'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Sinalizavia (Municípios)','Sinaliza Via (Rodovias)']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Sinalizavia (Municípios)','Sinaliza Via (Rodovias)'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Terrayama - DER Araçuaí']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Terrayama - DER Araçuaí'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Radar SA Concessões','Radar SA']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Radar SA Concessões','Radar SA'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Construtora Castilho','Construtora Castilho (Ferrovias)','Construtora Castilho (Grupo Pátria)','Construtora Castilho (Prefeitura Curitiba)']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name__in=['Construtora Castilho','Construtora Castilho (Ferrovias)','Construtora Castilho (Grupo Pátria)','Construtora Castilho (Prefeitura Curitiba)'],
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    elif company in ['Entrevias']:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            Q(user__email__icontains='afirma')|Q(user__email__icontains='ajr')|Q(user__username__icontains='ajr'),
            company__name__icontains=company,
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    else:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            company__name=company,
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    
    users_permissions = permissoes_history_user(historicoUsuarios,data)[0] if len(permissoes_history_user(historicoUsuarios,data)[0]) > 0 else 0
    cnpj=Company.objects.get(name=company).cnpj.replace('.', '').replace('/', '').replace('-', '')
    if users_permissions != 0:
        df2 = pd.DataFrame(data=users_permissions,columns=['Usuários ativos'])
        with pd.ExcelWriter('Usuários Ativos Financeiro.xlsx', mode='a',if_sheet_exists='replace') as writer:
            df2.to_excel(writer,sheet_name=cnpj,na_rep="-")

  0%|          | 0/5 [00:00<?, ?it/s]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalUserInCompany.history_date received a naive datetime (2024-03-31 03:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [2]:
Company.objects.filter(created_at='2024')

FieldError: Cannot resolve keyword 'created_at' into field. Choices are: accessrequest, active, additional_controls, cnpj, company_approval_flows, company_collects, company_construction_plants, company_constructions, company_daily_report_equipment, company_daily_report_signaling, company_daily_report_vehicles, company_daily_report_workers, company_dailyreports, company_dashboards, company_data_series, company_email_events, company_emails, company_equipment, company_export_requests, company_external_teams, company_firms, company_goal, company_group, company_group_id, company_jobasyncbatchs, company_log, company_logs, company_material_items, company_measurement, company_multipledailyreports, company_pending_procedures_exports, company_pushs, company_quality_assays, company_quality_samples, company_record_panels, company_reportinginreportingasyncbatchs, company_roads, company_service, company_specs, company_status_specs, company_subcompanies, company_syncs, company_table_data_series, comp

In [45]:
historicoUsuarios=UserInCompany.history.model.objects.filter(
            Q(user__email__icontains='afirma')|Q(user__email__icontains='ajr')|Q(user__username__icontains='ajr'),
            company__name__icontains="Entrevias",
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')

In [46]:
users_permissions = permissoes_history_user(historicoUsuarios,data)[0]
users_permissions

['Isabelly AJR',
 'Rogério Buchala',
 'Cassio Borges',
 'Luciano Teixeira',
 'Osmar Duprat',
 'Marcio Celestino']